# **모델 후보 2 테스트**

---

## 필요 라이브러리 import 과정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tqdm
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00


In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.0 MB/s eta 0:00:00


In [4]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, AutoTokenizer, pipeline
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import re
import json
import random
from random import sample
from tqdm import tqdm
import string

## 데이터 오픈 함수

In [5]:
# Custom Dataset 설정 (학습용 데이터를 설정한 형식대로 하나씩(or batch 크기만큼) 불러와줌)
class CustomDataset(Dataset):
    def __init__(self, datas):
        self.datas = datas
        self.len = len(datas)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.datas[idx]


def get_dataset(data):
    sentences = []
    labels = []
    for i in data:
        sentences.append(i['sentence'])
        labels.append(i['label'])

    datas = []
    for a, b in zip(sentences, labels):
        datas.append((a, b))

    return CustomDataset(datas)

### 데이터 경로 지정 단계

**▼ data : { sentence : pred_label } 형태로 저장된 파일 불러오기 -> dataset = get_dataset(data)**

In [14]:
# 사용할 데이터 설정 (보통 파일 형태로 불러옴 json.open() 등)
data = [{"sentence": "이새의 아들은 몇 명이며 다윗은 몇 번째 아들인가요",
         "label": 2},
        {"sentence": "임시보관중인 메일좀 모두 삭제 할 수 있니",
         "label": 4}]
# 데이터를 CustomDataset 형태로 불러옴
dataset = get_dataset(data)
# dataloader 설정 (batch_size, shuffle 등)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

## 모델 설정 단계

In [7]:
!nvidia-smi

Wed Sep  6 07:41:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0" <- 코랩에서는 GPU할당하려면 주석처리 해야함
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

In [9]:
device = "cuda:0"

모델 버전 : beomi/KoAlpaca-Polyglot-5.8B

( ※ 12.8b 사용시 메모리 부족으로 터짐 )

In [10]:
# 모델이름 설정
model_name = 'beomi/KoAlpaca-Polyglot-5.8B'
# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)

model.eval()
# pipeline 설정
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=model_name,
    device=0,
)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

## Template 수정 단계

In [15]:
# 문자열 template 만들기
# 세부 프롬프트는 최대한 덜 손대고 출력값을 정돈하는 방식으로 수정하기

prompt_tpl = string.Template('''### 지시 ###
아래 문장을 키워드 위주로 요약해주세요.

### 맥락 ###
키워드 위주 요약은 조사를 최소한으로 사용하여, 단어 3개 내로 문장의 의도나 사실을 요약하여 다시 적는 것입니다.
해당 문장은 $label입니다.
$typecontext

### 예시 ###
$exemplars
요약 전: $query → 요약 후:''')

**txtlabel 정의 단계 및 예시 추가**

* 논문 3.2.3 부터 4.2 까지 읽고 명확한 의미의 문장이 있으면 추가

* 없으면 사전적 정의나 프롬프트가 알아듣는 말로 수정하기

In [18]:
# 사용할 데이터 설정 (보통 파일 형태로 불러옴 json.open() 등)
data = [{"sentence": "이번 태풍 경로가 어떤지 봐줘",
         "label": 2},
        {"sentence": "매달 첫째주 수요일마다 건강검진 일정 입력해줘",
         "label": 4}]
# 데이터를 CustomDataset 형태로 불러옴
dataset = get_dataset(data)
# dataloader 설정 (batch_size, shuffle 등)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
# dataloader에서 하나씩 불러와서 모델 돌리기
for i in dataloader:
    query, label = i
    query = query[0]
    txtlabel = ""

    # label별 설명 추가하기
    if label == 0:
        txtlabel = "여부"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전: 내일 우산을 안 챙겨도 되겠니 → 요약 후: 내일 비 여부 '

    if label == 1:
        txtlabel = "선택"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전: 주식 투자는 운입니까 실력입니까 → 요약 후: 투자가 운인지 실력인지  '

    if label == 2:
        txtlabel = "질문"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전: 밥이 언제쯤 될 지 알려 줄래? → 요약 후: 밥 되는 시간'

    if label == 3:
        txtlabel = "금지"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전: 음주운전은 절대로 하면 안 돼 알았지 → 요약 후: 음주운전하지 않기  '

    if label == 4:
        txtlabel = "요구"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전: 매주 화요일과 목요일 오후 여섯시 폴댄스 일정 추가해줘  → 요약 후: 폴댄스 일정 추가하기 '

    if label == 5:
        txtlabel = "강한 요구"
        typecontext_str = "입력 문장의 질문 의도를 파악하여, 대답해야 하는 내용을 위주로 3단어 이내로 요약합니다."
        fewshot_str = '요약 전:   → 요약 후:   '

    prompt = prompt_tpl.substitute(query=query, label=txtlabel, typecontext=typecontext_str, exemplars=fewshot_str)

    ans = pipe(
            prompt,
            do_sample=False,
            max_new_tokens=512,
            temperature=0.1,
            top_p=0.9,
            return_full_text=False,
            pad_token_id=pipe.tokenizer.eos_token_id,
            eos_token_id=2,
        )                                                       # 프롬프트를 모델에 넣어서 돌림 (prompt이외에는 설정한 파라미터들)

    # 후처리
    res = ans[0]['generated_text']
    # res의 앞뒤 \n 없애기
    proc_res = res.strip().split('\n')
    if len(res) >= 1:
        proc_res = proc_res[0]

    print(prompt, proc_res)
    print(f"\n실제 생성된 문장 전체: [{res}]\n\n\n")

---

성공 사례 1

{"sentence" : "이번 태풍 경로가 어떤지 봐줘", "label": 2 }

실제 생성된 문장 전체: 이번 태풍의 진로

\

{"sentence" : "매달 첫째주 수요일마다 건강검진 일정 입력해줘", "label": 4}

실제 생성된 문장 전체: 건강검진 일정 추가하기

